In [1]:
import numpy as np
import re
import pandas as pd
from tqdm.notebook import tqdm
from datasets import load_dataset
import umap
import altair as alt
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer
from typing import List
import enum

from langchain_community.llms import Ollama
from langchain.output_parsers.regex_dict import RegexDictParser
from langchain.output_parsers import PydanticOutputParser
from langchain_core.messages import HumanMessage, SystemMessage, ChatMessage
from langchain.prompts import ChatPromptTemplate, PromptTemplate
from pydantic import BaseModel, Field, validator, create_model
from openai import AsyncOpenAI, OpenAI
import asyncio
import os

import requests
import json

import itertools
from copy import deepcopy
from tqdm.notebook import tqdm, trange
from sklearn.cluster import KMeans

import umap.umap_ as umap
#import umap
import hdbscan

from typing import Literal, Union
from pydantic.config import ConfigDict

from src.models import *

from src.utilities import *
from src.bubble import *

import openai
import instructor

#openai.api_key = os.environ["OPENAI_API_KEY"]


In [4]:
with open('Prompts/prompt_company_informations.txt') as f:
    template = PromptTemplate.from_template(f.read())
prompt_company_informations = template.invoke(context)

class CompanyContext(BaseModel):
    context: str = Field(description="Contexte de l'entreprise.")

    def __str__(self):
        return self.context

res = get_analysis(client, prompt_company_informations, CompanyContext)
context['context'] = res.context
print(res)

Darty est une entreprise française spécialisée dans la distribution d'électroménager, d'informatique et d'électronique grand public. Elle a été fondée en 1957 par Natan Darty et est aujourd'hui présente dans plusieurs pays européens. En 2016, Darty a été rachetée par la Fnac, créant ainsi un grand groupe français de distribution de produits culturels et techniques. Plus récemment, en 2021, le groupe Fnac Darty a annoncé son intention de se rapprocher de l'entreprise britannique Dixons Carphone, ce qui pourrait donner naissance à un leader paneuropéen de la distribution de produits technologiques. Parmi les concurrents de Darty, on peut citer Boulanger, But, et Conforama.


In [25]:
with open('Prompts/prompt_categories_creation.txt') as f:
    template = PromptTemplate.from_template(f.read())
prompt_categories_creation = template.invoke(context)

class InsightSubCategory(BaseModel):
    name: str = Field(description="Nom de la sous-catégorie.")
    description: str = Field(description="Phrase de description de la sous-catégorie.")
    id: str
    def __str__(self):
        return '\n- ' + self.name + ": " + self.description

class InsightCategory(BaseModel):
    name: str = Field(description="Nom de la catégorie.")
    #description: str = Field(description="Phrase de description de la catégorie.")
    sub_categories: List[InsightSubCategory] = Field(description="Sous-catégories.")
    id: str

    def __str__(self):
        return self.name + " : " + ''.join([str(x) for x in self.sub_categories])

class InsightCategories(BaseModel):
    categories: List[InsightCategory] = Field(description="Catégories parmi lesquels les insights pourront être classifiées.")

    def __str__(self):
        return '\n\n'.join([str(x) for x in self.categories])

res = get_analysis(client, prompt_categories_creation, InsightCategories)
print(res)

Satisfaction globale : 
- Note globale: Indique le niveau de satisfaction global du client vis-à-vis de Darty.
- Commentaire général: Permet au client de s'exprimer librement sur son expérience chez Darty.

Expérience d'achat : 
- Navigabilité du site: Évalue la facilité de navigation sur le site web de Darty.
- Disponibilité des produits: Évalue la disponibilité des produits recherchés par le client sur le site web de Darty.
- Processus de commande: Évalue la rapidité et l'efficacité du processus de commande sur le site web de Darty.
- Livraison des produits: Évalue la rapidité et l'efficacité de la livraison des produits commandés sur le site web de Darty.

Qualité des produits : 
- Qualité globale des produits: Évalue la qualité des produits proposés par Darty.
- Précision de la description: Évalue la conformité des produits commandés par rapport à leur description sur le site web de Darty.
- Fiabilité et durabilité: Évalue la fiabilité et la durabilité des produits proposés par Dar

In [22]:
for categorie in tqdm(res.categories):
    categorie.id = bubble_client.create(
        "Category",
        {
            "Company": COMPANY_ID,
            "Name": categorie.name,
            "Project": PROJECT_ID,
        })
    
    for sub_categorie in categorie.sub_categories:
        sub_categorie.id = bubble_client.create(
            "SubCategory",
            {
                "Company": COMPANY_ID,
                "Parent_category": categorie.id,
                "Name": sub_categorie.name,
                "Project": PROJECT_ID,
                "Description": categorie.description,
            })